# 🎙️ ASR Project Demo
Демонстрация работы системы распознавания речи (CTC).
**Инструкция:** Запустите ячейки по порядку. Во второй ячейке вставьте ссылку на ZIP-архив с Google Drive (должен содержать папки `audio` и `transcriptions`).

In [ ]:
# 1. 🛠️ Установка и Настройка окружения
import os

# 1. Клонируем репозиторий
if not os.path.exists('speech-recognition-ctc'):
    !git clone https://github.com/Olga57/speech-recognition-ctc.git

# 2. Переходим в папку проекта
%cd speech-recognition-ctc

# 3. Устанавливаем зависимости
!pip install -q -r requirements.txt
!pip install -q gdown
!apt-get install -y libsndfile1 ffmpeg > /dev/null

# 4. Настраиваем пути
import sys
sys.path.append(os.path.join(os.getcwd(), 'src'))
os.environ['PYTHONPATH'] = os.getcwd()

print('✅ Репозиторий клонирован, окружение готово.')

In [ ]:
# 2. 📂 Тестирование на вашем датасете (Google Drive)
import gdown
import os

url = input('Вставьте ссылку на Google Drive (ZIP архив с папками audio и transcriptions): ')

if 'drive.google.com' in url:
    # 1. Скачивание
    file_id = url.split('/d/')[-1].split('/')[0]
    dl_url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(dl_url, 'user_data.zip', quiet=False)
    
    # 2. Распаковка
    !unzip -q -o user_data.zip -d user_dataset
    
    # 3. Авто-поиск корня
    ds_root = 'user_dataset'
    for r, d, f in os.walk('user_dataset'):
        if 'audio' in d: 
            ds_root = r
            break
    
    print(f'📂 Данные найдены в: {ds_root}')

    # 4. ИНФЕРЕНС
    print('🚀 Запускаем распознавание...')
    !python inference.py \
        dataset=custom \
        +dataset.custom_root="{ds_root}" \
        +checkpoint="weights/best.pt" \
        +device="cuda" \
        +decode="beam" \
        +dataset.train_source=null

    # 5. МЕТРИКИ
    print('\n📊 Результаты (WER/CER):')
    if os.path.exists("predictions"):
        !python calc_metrics.py \
            --ref_dir "{ds_root}/transcriptions" \
            --hyp_dir "predictions"
        
        print('\n📝 Пример распознавания:')
        !head -n 2 predictions/*.txt
    else:
        print('❌ Ошибка: Папка predictions пуста.')

else:
    print('⚠️ Введите корректную ссылку.')